# Package & Definition

In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchmetrics
from torchmetrics import MeanSquaredError
from sklearn.impute import KNNImputer
import scripts
from functools import lru_cache
import optuna
from sklearn.metrics import pairwise_distances
from sklearn.exceptions import DataConversionWarning
import warnings
from torch.utils.data import Dataset
from datetime import datetime
import uuid 
import time
# 경고 무시 설정
warnings.filterwarnings("ignore", category=DataConversionWarning)
warnings.filterwarnings("ignore", category=RuntimeWarning)
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=pd.errors.DtypeWarning)
warnings.simplefilter("ignore", DeprecationWarning)

/home/kim14/.conda/envs/jupyter/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# dataset for sim_matrix
class OmicsDataset_dict(Dataset): 
    def __init__(self, omic_dict, drug_dict, data): 
        self.omic_dict = omic_dict
        self.drug_dict = drug_dict
        self.cell_mapped_ids = {key:i for i, key in enumerate(self.omic_dict.keys())}
        # omic_dict의 키를 고유한 인덱스로 매핑
        # enumerate는 키들을 순서대로 열거하여 (인덱스, 키) 형태의 튜플로 반환
        # 딕셔너리 컴프레헨션: 각 키를 key로, 각 키의 인덱스를 i로 사용하여 {key:i}형태로 매핑된 딕셔너리 만듬.
        self.drug_mapped_ids = {key:i for i, key in enumerate(self.drug_dict.keys())}
        self.data = data
        
    def __len__(self):
        return len(self.data)
        
    def __getitem__(self, idx): # idx = train_data
        instance = self.data.iloc[idx] 
        cell_id = instance.iloc[0]
        drug_id = instance.iloc[1]
        target = instance.iloc[2]
        
        #omics_data = { # usage of dictionary here causes a problem or crash with collate_fn function in Dataloader 
        #    cell_id : {
        #        data_type: self.omic_dict[cell_id][data_type] for data_type in self.omic_dict[cell_id].keys()
        #    }
        #}
        
        return (torch.cat([self.omic_dict[cell_id][modality] for modality in self.omic_dict[cell_id].keys()]), 
                self.drug_dict[drug_id],
                torch.Tensor([target]),
                torch.Tensor([self.cell_mapped_ids[cell_id]]),
                torch.Tensor([self.drug_mapped_ids[drug_id]])) 

In [3]:
# MDA model
class MultimodalAutoencoder(nn.Module):
    def __init__(self, input_dims,  hidden_dim_encoders = 150, embed_dim = 75, fusion_dim = 150, dropout_encoders = 0.2):
        # get input as a dictionary
        super(MultimodalAutoencoder, self).__init__()
        # EEEEEEEEEEncoder
        self.input_dims = input_dims
        self.num_modalities = len(input_dims)
        self.do = nn.Dropout(dropout_encoders)

        self.omics_encoder = nn.ModuleList([
            nn.Sequential(
                nn.Linear(mod_input_dim, hidden_dim_encoders), # input 
                nn.ReLU(),
                nn.Linear(hidden_dim_encoders, embed_dim) # encoder hidden layer: 150, 75 as the value from the paper. so we start from this 
            )                                 # I dont get why they used 150, 75 for dimension, but we can tune it later
            for mod_input_dim in self.input_dims
        ])
        # fused latent feature 
        self.fusion_layer = nn.Sequential( # I think we need a fusion layer here, to combine the data modalities
            nn.Linear(embed_dim * self.num_modalities, fusion_dim),
            nn.ReLU(),
            nn.Linear(fusion_dim, embed_dim) # This concatenate latent features of all omics data, and fusion them and make its dim final latent dim
        )                                     # This is the only way I can think of to fuse omics data
        # decoder
        self.omics_decoder = nn.Sequential(
            nn.Linear(embed_dim, hidden_dim_encoders),
            nn.ReLU(),
            nn.Linear(hidden_dim_encoders, sum(input_dims))
        )
        # I actually dont understand this step in paper. they said that decoder has symmetric structure as encoder,
        # but the data after MDA they provided, has weird dimension(363x90) which makes no sense. this is the point that i cant understand

    def forward(self, x):
        latent_features = [] # get dictionary as an input 
        start_idx = 0
        for i, encoder in enumerate(self.omics_encoder):
            mod_input_dim = self.input_dims[i]
            x_modality = x[:, start_idx:start_idx + mod_input_dim]
            latent_features.append(encoder(self.do(x_modality)))
            start_idx += mod_input_dim
            
        latent_fused = torch.cat(latent_features, dim=1)
        latent_final = self.fusion_layer(latent_fused)
        decoded = self.omics_decoder(latent_final)
        return decoded, latent_final

In [4]:
# Main resnet model
class ResNet(nn.Module):
    def __init__(self, embed_dim=256, hidden_dim=1024, dropout=0.1, n_layers=6, norm="layernorm"):
        super().__init__()
        self.mlps = nn.ModuleList()
        
        # Determine normalization layer
        if norm == "layernorm":
            norm_layer = nn.LayerNorm
        elif norm == "batchnorm":
            norm_layer = nn.BatchNorm1d
        else:
            norm_layer = nn.Identity
        
        # Create MLP layers
        for _ in range(n_layers):
            self.mlps.append(
                nn.Sequential(
                    nn.Linear(embed_dim, hidden_dim),
                    norm_layer(hidden_dim),
                    nn.ReLU(),
                    nn.Dropout(dropout),
                    nn.Linear(hidden_dim, embed_dim)
                )
            )
        
        self.lin = nn.Linear(embed_dim, 1)  # Output layer

    def forward(self, x):
        for layer in self.mlps:
            
            x = (layer(x) + x) / 2  # Residual connection
        return self.lin(x)


class Main_model(nn.Module):
    def __init__(self, embed_dim=256, hidden_dim=1024, dropout=0.1, n_layers=6, norm="layernorm", 
                 dropout_omics=0.4, dropout_omics_finetuning=0.4):
        super().__init__()
        
        # ResNet as backbone
        self.resnet = ResNet(embed_dim, hidden_dim, dropout, n_layers, norm)
        
        # Modified embed_d: Two-layer MLP with dropout after ReLU
        self.embed_d = nn.Sequential(
            nn.LazyLinear(hidden_dim),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim, embed_dim)
        )
        
        # Modified embed_c: Two-layer MLP with dropout after ReLU and before first Linear Layer
        self.embed_c = nn.Sequential(
            nn.Dropout(dropout_omics_finetuning),
            nn.LazyLinear(hidden_dim),
            nn.ReLU(),
            nn.Dropout(dropout_omics),
            nn.Linear(hidden_dim, embed_dim)
        )

    def forward(self, c, d):
        # Combine embedded inputs and pass through ResNet
        return self.resnet(self.embed_d(d) + self.embed_c(c))


In [5]:
# get Data function
@lru_cache(maxsize=None)
def get_data_corr(n_fold = 0, fp_radius = 2, transform_into_corr = True, typ = ["rnaseq", "mutations", "cnvs"],
                  #reconstructed = None
                 ):
    # drug
    smile_dict = pd.read_csv("data/smiles.csv", index_col=0)
    fp = scripts.FingerprintFeaturizer(R = fp_radius)
    drug_dict = fp(smile_dict.iloc[:, 1], smile_dict.iloc[:, 0])
    
    # loading all datasets
    driver_genes = pd.read_csv("data/driver_genes.csv").loc[:, "symbol"].dropna()

    rnaseq = pd.read_csv("data/rnaseq_normcount.csv", index_col=0)
    driver_columns = rnaseq.columns.isin(driver_genes)
    filtered_rna = rnaseq.loc[:, driver_columns]
    
    proteomics = pd.read_csv("data/proteomics.csv", index_col=0)
    
    mutation = pd.read_csv("data/binary_mutations.csv")
    mutation.columns = mutation.iloc[0]
    mutation = mutation.iloc[2:,:].set_index("gene_symbol")
    driver_columns = mutation.columns.isin(driver_genes)
    filtered_mut = mutation.loc[:, driver_columns]
    filtered_mut = filtered_mut.astype(float)

    methylations = pd.read_csv("data/methylations.csv",index_col = 0).sort_index(ascending = True)

    cnvs = pd.read_csv("data/copy_number_variations.csv",index_col= 0)

    # concatenate all dataset 
    # inner join based on index: model_ids with NaN are automatically filtered out 
    data_concat = pd.concat([filtered_rna, proteomics, filtered_mut, methylations, cnvs], axis=1, join='inner')
    
    
    # Filter data by common indices in all modalities
    filtered_rna = filtered_rna[filtered_rna.index.isin(data_concat.index)]
    proteomics = proteomics[proteomics.index.isin(data_concat.index)]
    filtered_mut = filtered_mut[filtered_mut.index.isin(data_concat.index)]
    methylations = methylations[methylations.index.isin(data_concat.index)]
    cnvs = cnvs[cnvs.index.isin(data_concat.index)]
    
    # Initialize cell_dict
    cell_dict = {}

    if not transform_into_corr : #and reconstructed is None:

        dims = []
        if "rnaseq" in typ:
            dims.append(filtered_rna.shape[1])
        if "proteomics" in typ:
            dims.append(proteomics.shape[1])
        if "mutations" in typ:
            dims.append(filtered_mut.shape[1])
        if "methylations" in typ:
            dims.append(methylations.shape[1])
        if "cnvs" in typ:
            dims.append(cnvs.shape[1])
        
        for cell in data_concat.index:
            # Initialize a sub-dictionary for each cell
            concatenated_data = []
            
            # Add data for each type specified in typ
            if "rnaseq" in typ:
                concatenated_data.append(filtered_rna.loc[cell].to_numpy())
            if "proteomics" in typ:
                concatenated_data.append(proteomics.loc[cell].to_numpy())
            if "mutations" in typ:
                concatenated_data.append(filtered_mut.loc[cell].to_numpy())
            if "methylations" in typ:
                concatenated_data.append(methylations.loc[cell].to_numpy())
            if "cnvs" in typ:
                concatenated_data.append(cnvs.loc[cell].to_numpy())

            cell_dict[cell] = torch.Tensor(np.concatenate(concatenated_data))
            
#    if reconstructed is not None:
#        for cell_idx, cell in enumerate(data_concat.index):
#            # cell_dict에 reconstructed 텐서의 각 행(cell 데이터) 저장
#            cell_dict[cell] = reconstructed[cell_idx]
        

    # GDSC
    GDSC1 = pd.read_csv("data/GDSC1.csv", index_col=0)
    # default, remove data where lines or drugs are missing:
    data = GDSC1.query("SANGER_MODEL_ID in @data_concat.index & DRUG_ID in @drug_dict.keys()")
    unique_cell_lines = data.loc[:, "SANGER_MODEL_ID"].unique()

    np.random.seed(420) # for comparibility, don't change it!
    np.random.shuffle(unique_cell_lines)
    folds = np.array_split(unique_cell_lines, 10)
    #test_lines = folds[0] 
    train_idxs = list(range(10))
    train_idxs.remove(n_fold)
    np.random.seed(420)
    validation_idx = np.random.choice(train_idxs)
    train_idxs.remove(validation_idx)
    train_lines = np.concatenate([folds[idx] for idx in train_idxs])
    validation_lines = folds[validation_idx]
    test_lines = folds[n_fold] 

        # no change needed, query works fine with some missing
    train_data = data.query("SANGER_MODEL_ID in @train_lines")
    validation_data = data.query("SANGER_MODEL_ID in @validation_lines")
    test_data = data.query("SANGER_MODEL_ID in @test_lines")
    
    if transform_into_corr: #and reconstructed is None:
        # train, val, test among filtered data
        # these are valid train_, val_ and test_data index
        
        
        n_train = len(train_lines)  
        n_val = len(validation_lines)      
        n_test = len(test_lines)
        
        # Precompute similarity matrices for each data type
        similarity_matrices = {}
        dims = []
        
        if "rnaseq" in typ:
            exp_com = np.corrcoef(np.vstack([filtered_rna.loc[train_lines], 
                                             filtered_rna.loc[validation_lines], 
                                             filtered_rna.loc[test_lines]]), rowvar=True)
            train = exp_com[:n_train, :n_train]
            val = exp_com[n_train:n_train+n_val, :n_train]
            test = exp_com[n_train+n_val:, :n_train]
            sim_combined = np.vstack([train, val, test])
            similarity_matrices["rnaseq"] = sim_combined
            dims.append(len(similarity_matrices["rnaseq"][0]))
        
        if "proteomics" in typ:
            prot_com = np.corrcoef(np.vstack([proteomics.loc[train_lines], 
                                              proteomics.loc[validation_lines], 
                                              proteomics.loc[test_lines]]), rowvar=True)
            train = prot_com[:n_train, :n_train]
            val = prot_com[n_train:n_train+n_val, :n_train]
            test = prot_com[n_train+n_val:, :n_train]
            sim_combined = np.vstack([train, val, test])
            similarity_matrices["proteomics"] = sim_combined
            dims.append(len(similarity_matrices["proteomics"][0]))
        
        if "mutations" in typ:
            train_snp = filtered_mut.loc[train_lines].astype(bool)
            val_snp = filtered_mut.loc[validation_lines].astype(bool)
            test_snp = filtered_mut.loc[test_lines].astype(bool)
            
            train = 1 - pairwise_distances(train_snp.values, train_snp.values, metric="jaccard")
            val = 1 - pairwise_distances(val_snp.values, train_snp.values, metric="jaccard")
            test = 1 - pairwise_distances(test_snp.values, train_snp.values, metric="jaccard")
    
            sim_combined = np.vstack([train, val, test])
            similarity_matrices["mutations"] = sim_combined
            dims.append(len(similarity_matrices["mutations"][0]))
        
        if "methylations" in typ:
            methyl_com = np.nan_to_num(np.corrcoef(np.vstack([methylations.loc[train_lines], 
                                                methylations.loc[validation_lines], 
                                                methylations.loc[test_lines]]), rowvar=True))
            train = methyl_com[:n_train, :n_train]
            val = methyl_com[n_train:n_train+n_val, :n_train]
            test = methyl_com[n_train+n_val:, :n_train]
            sim_combined = np.vstack([train, val, test])
            similarity_matrices["methylations"] = sim_combined
            dims.append(len(similarity_matrices["methylations"][0]))
        
        if "cnvs" in typ:
            cnv_com = np.nan_to_num(np.corrcoef(np.vstack([cnvs.loc[train_lines], # nan-generation problem fixed 
                                             cnvs.loc[validation_lines], 
                                             cnvs.loc[test_lines]]), rowvar=True))
            train= cnv_com[:n_train, :n_train]
            val= cnv_com[n_train:n_train+n_val, :n_train]
            test= cnv_com[n_train+n_val:, :n_train]
            sim_combined = np.vstack([train, val, test])
            similarity_matrices["cnvs"] = sim_combined
            dims.append(len(similarity_matrices["cnvs"][0]))
            
        cell_dict = {}

        # 
        for cell in unique_cell_lines:
            cell_dict[cell] = {}
            for data_type in typ:
                sim_matrices = similarity_matrices[data_type]
                sim_tensor = torch.Tensor(sim_matrices)
                cell_idx = np.where(unique_cell_lines == cell)[0][0]
                cell_dict[cell][data_type] = sim_tensor[cell_idx]
    
        return (OmicsDataset_dict(cell_dict, drug_dict, train_data),
        OmicsDataset_dict(cell_dict, drug_dict, validation_data),
        OmicsDataset_dict(cell_dict, drug_dict, test_data),
        dims)

    return (scripts.OmicsDataset(cell_dict, drug_dict, train_data),
    scripts.OmicsDataset(cell_dict, drug_dict, validation_data),
    scripts.OmicsDataset(cell_dict, drug_dict, test_data),
    dims)

In [6]:
def chain_train_step(autoencoder, resnet, lamda, optimizer, loader, config, device):
    """
    Autoencoder와 ResNet을 동시에 학습하는 End-to-End Training Step
    """

    # 손실 함수 정의
    loss_fn_recon = nn.MSELoss()  # Autoencoder의 Reconstruction Loss
    loss_fn_pred = nn.MSELoss()   # ResNet의 Prediction Loss
    ls = []  # ResNet의 Loss 저장
    ls_recon = []  # Autoencoder의 Reconstruction Loss 저장

    autoencoder.train()  # Autoencoder 학습 모드
    resnet.train()  # ResNet 학습 모드
    
    for batch in loader:
        optimizer.zero_grad()

        # 1️⃣ Autoencoder Forward (Latent Feature 생성)
        reconstructed, latent_features = autoencoder(batch[0].to(device))  # (batch_size, latent_dim)
        
        # 2️⃣ ResNet Forward (Latent Feature + Drug 데이터)
        out = resnet(latent_features, batch[1].to(device))

        # 3️⃣ Loss 계산
        loss_recon = loss_fn_recon(reconstructed, batch[0].to(device))  # Autoencoder Loss
        loss_pred = loss_fn_pred(out.squeeze(), batch[2].to(device).squeeze())  # ResNet Loss

        # 4️⃣ Total Loss = Reconstruction Loss + Prediction Loss
        loss = lamda*loss_recon + (1-lamda)*loss_pred

        # 5️⃣ Backpropagation
        loss.backward() # 두 losses를 모두 backpropagation한다.
        torch.nn.utils.clip_grad_norm_(list(autoencoder.parameters()) + list(resnet.parameters()), config["optimizer"]["clip_norm"])
        optimizer.step()

        # 6️⃣ Loss 저장
        ls.append(loss_pred.item())
        ls_recon.append(loss_recon.item())

    return np.mean(ls), np.mean(ls_recon)


def train_auto_resnet_chain(config, train_dataset, lamda=0.5, validation_dataset=None, use_momentum=True, callback_epoch=None):
    train_loader = torch.utils.data.DataLoader(
        train_dataset,
        batch_size=config["optimizer"]["batch_size"],
        drop_last=True,
        shuffle=True
    )

    if validation_dataset is not None:
        val_loader = torch.utils.data.DataLoader(
            validation_dataset,
            batch_size=config["optimizer"]["batch_size"],
            drop_last=False,
            shuffle=False
        )

    autoencoder = MultimodalAutoencoder(
        input_dims=input_dims,
        hidden_dim_encoders=config["model"]["hidden_dim_encoders"],
        embed_dim=config["model"]["embed_dim"],
        fusion_dim=config["model"]["fusion_dim"],
        dropout_encoders=config["model"]["dropout_encoders"]
    )

    resnet = Main_model(
        embed_dim=config["model"]["embed_dim"],
        hidden_dim=config["model"]["hidden_dim"], 
        dropout=config["model"]["dropout"], 
        n_layers=config["model"]["n_layers"],  
        dropout_omics=config["model"]["dropout_omics"], 
        dropout_omics_finetuning=config["model"]["dropout_omics_finetuning"],
        norm=config["model"]["norm"]
    )

    optimizer = torch.optim.Adam(list(autoencoder.parameters()) + list(resnet.parameters()), config["optimizer"]["learning_rate"])
    lr_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.5, patience=5)
    early_stop = scripts.EarlyStop(config["optimizer"]["stopping_patience"])
    device = torch.device(config["env"]["device"])
    autoencoder.to(device)
    resnet.to(device)
    autoencoder.train()
    resnet.train()

    metrics = torchmetrics.MetricTracker(torchmetrics.MetricCollection({
        "R_cellwise_residuals": scripts.GroupwiseMetric(
            metric=torchmetrics.functional.pearson_corrcoef,
            grouping="drugs",
            average="macro",
            residualize=True
        ),
        "R_cellwise": scripts.GroupwiseMetric(
            metric=torchmetrics.functional.pearson_corrcoef,
            grouping="cell_lines",
            average="macro",
            residualize=False
        ),
        "MSE": torchmetrics.MeanSquaredError()
    }))
    metrics.to(device)

    num_epochs = config["env"]["max_epochs"]
    
    for epoch in range(num_epochs):
        ### 1️⃣ Training Step ###
        train_loss_pred, train_loss_recon = chain_train_step(autoencoder, resnet, lamda, optimizer, train_loader, config, device)
        total_train_loss = train_loss_pred + train_loss_recon
            
        # Learning Rate Scheduler 업데이트
        lr_scheduler.step(total_train_loss)

        ### 2️⃣ Validation Step ###
        if validation_dataset is not None:
            with torch.no_grad():
                validation_metrics = evaluate_step(autoencoder, resnet, val_loader, metrics, device)
    
                # Momentum을 사용한 Validation Metric 업데이트
                if epoch > 0 and use_momentum:
                    val_target = 0.2 * val_target + 0.8 * validation_metrics['R_cellwise_residuals']
                else:
                    val_target = validation_metrics['R_cellwise_residuals']
        else:
            val_target = None
        
        # 로그 출력
        if callback_epoch is None:
            print(f"Epoch {epoch+1}/{num_epochs} | Train Loss: {total_train_loss} | Validation R: {val_target}")
        else:
            callback_epoch(epoch, val_target)

        # Early Stopping 체크
        if early_stop(total_train_loss):
            print("⏹ Early Stopping Triggered. Stopping Training.")
            break

    print("✅ Training Complete!")

    return val_target, autoencoder, resnet
    

    

In [7]:
# evaluation step
def evaluate_step(autoencoder, resnet, loader, metrics, device, save_predictions=False, model_name = "model", dataset_name = "dataset"):
    metrics.increment()
    autoencoder.eval()  # Autoencoder Evaluation Mode
    resnet.eval()  # ResNet Evaluation Mode

    predictions = {"cell_line": [], "drug_id": [], "prediction": [], "target": []}

    for batch in loader:
        with torch.no_grad():
            _, latent_features = autoencoder(batch[0].to(device))  # Autoencoder에서 Latent Feature 생성
            out = resnet(latent_features, batch[1].to(device))  # ResNet 예측

            metrics.update(out.squeeze(),
                           batch[2].to(device).squeeze(),
                           cell_lines=batch[3].to(device).squeeze(),
                           drugs=batch[4].to(device).squeeze())

            # 결과 저장
            predictions["cell_line"].extend(batch[3].squeeze().tolist())  
            predictions["drug_id"].extend(batch[4].squeeze().cpu().tolist())    
            predictions["prediction"].extend(out.squeeze().tolist()) 
            predictions["target"].extend(batch[2].squeeze().cpu().tolist())    

    metrics_dict = {it[0]: it[1].item() for it in metrics.compute().items()}

    # Save predictions to a CSV file if required
    if save_predictions:
        df = pd.DataFrame(predictions)
        filename = generate_filename(model_name, dataset_name, extension="csv")
        df.to_csv("results/" + filename, index=False)
        print(f"Predictions saved to: results/{filename}")

    return metrics_dict

def generate_filename(model_name="model1", dataset_name="dataset", extension="csv"):
    time = datetime.now().strftime("%Y%m%d_%H:%M:%S")
    unique_id = uuid.uuid4()
    filename = f"pred_{model_name}_{dataset_name}_{time}_{unique_id}.{extension}"
    return filename

In [8]:
#Best Hyperparameters: {'embed_dim': 493, 'hidden_dim_encoders': 868, 'fusion_dim': 592, 'dropout_encoders': 0.2670033232985919, 'pre_training_epochs': 160, 'hidden_dim': 1982, 'dropout': 0.2965716546354523, 'n_layers': 5, 'dropout_omics': 0.8626326149215306, 'dropout_omics_finetuning': 0.5489897453017749, 'batch_size': 261, 'clip_norm': 10, 'learning_rate': 3.3761769665931243e-06, 'lr_pretraining': 1.435539966576822e-06, 'pre_batch_size': 81}
# new best config
# config
config = {"features" : {"fp_radius":2,
                        "use_correlation_representation": True,
                        "num_modalities": 4},
          "optimizer": {"batch_size": 261,
                        "clip_norm":10,
                        "learning_rate": 3.3761769665931243e-06,
                        "stopping_patience":15,
                        "pre_batch_size": 81,
                        "lr_pretraining": 1.435539966576822e-06},
          "model":{"embed_dim":493, # shared
                 "hidden_dim":1982, 
                 "dropout":0.2965716546354523, 
                 "n_layers": 5, 
                 "norm": "batchnorm", 
                 "hidden_dim_encoders": 868, # ENCODER
                 "fusion_dim": 592, # ENCODER
                 "dropout_encoders": 0.2670033232985919,
                 "dropout_omics": 0.8626326149215306, # second
                 "dropout_omics_finetuning": 0.5489897453017749, # first
                 "pre_training_epochs": 160}, 
         "env": {"fold": 0,  
                "device":"cuda:0", 
                 "max_epochs": 100, 
                 "search_hyperparameters":False}} 

# Best Hyperparameters: {'embed_dim': 215, 'hidden_dim_encoders': 1330, 'fusion_dim': 505, 'dropout_encoders': 0.3626415306459327, 'pre_training_epochs': 184, 'pre_batch_size': 474, 'lr_pretraining': 3.9761097548681355e-06, 'dropout_omics': 0.5398956377457799, 'dropout_omics_finetuning': 0.42563415722955866, 'learning_rate': 0.0006230634247016341, 'batch_size': 469}

# Best Hyperparameters: {'embed_dim': 258, 'hidden_dim_encoders': 1236, 'fusion_dim': 447, 'dropout_encoders': 0.44181165428653146, 'pre_training_epochs': 190, 'pre_batch_size': 460, 'lr_pretraining': 2.480792529122e-06, 'dropout_omics': 0.5666807093758374, 'dropout_omics_finetuning': 0.3833537931230297, 'learning_rate': 0.0008108830765296783, 'batch_size': 408}

# Model training 
## Get data

In [9]:
start_time = time.time()
# data loading with raw data
train_dataset, val_dataset, test_dataset, input_dims= get_data_corr(n_fold = config["env"]["fold"],
                                                           fp_radius = config["features"]["fp_radius"], 
                                                           transform_into_corr = False,
                                                           typ = ("rnaseq", "proteomics", "mutations", "methylations"))

[01:26:40] DEPRECATION WARNING: please use MorganGenerator
[01:26:40] DEPRECATION WARNING: please use MorganGenerator
[01:26:40] DEPRECATION WARNING: please use MorganGenerator
[01:26:40] DEPRECATION WARNING: please use MorganGenerator
[01:26:40] DEPRECATION WARNING: please use MorganGenerator
[01:26:40] DEPRECATION WARNING: please use MorganGenerator
[01:26:40] DEPRECATION WARNING: please use MorganGenerator
[01:26:40] DEPRECATION WARNING: please use MorganGenerator
[01:26:40] DEPRECATION WARNING: please use MorganGenerator
[01:26:40] DEPRECATION WARNING: please use MorganGenerator
[01:26:40] DEPRECATION WARNING: please use MorganGenerator
[01:26:40] DEPRECATION WARNING: please use MorganGenerator
[01:26:40] DEPRECATION WARNING: please use MorganGenerator
[01:26:40] DEPRECATION WARNING: please use MorganGenerator
[01:26:40] DEPRECATION WARNING: please use MorganGenerator
[01:26:40] DEPRECATION WARNING: please use MorganGenerator
[01:26:40] DEPRECATION WARNING: please use MorganGenerat

여기서 Autoencoder의 evaluation은 수행하지 않는다. Autoencoder를 단독으로 평가하는 것이 Test distribution과 맞지 않을 수 있기 때문이다. 여기서 이걸 먼저 평가하게 되면 Test set의 정보가 노출되어 편향된 결과가 나타날 수 있다. 

또한, 단순 Reconstruction Loss로 이를 중간에 평가하는 것이 애초에 의미가 없을 가능성이 높다. 

그렇다면, 두 모델을 따로 트레이닝 할 것인지, 한번에 트레이닝 할 것인지 정해야겠다. 

1. Autoencoder pretraining -> encoder frozen, used as feature extractor -> Resnet training
   - Autoencoder가 단순 Reconstruction Loss로 학습되므로, ResNet에 필요한 Feature를 충분히 학습하지 못할 가능성이 있음.
3. Autoencoder -> latent feature -> Resnet
   - 설계가 복잡해짐.
   - 두 모델을 한번에 트레이닝하므로, 학습이 불안정할 수 있음. 


둘다 해볼건데, 여기서는 2번을 할거다. 간략적인 플로우는 다음과 같다.

- autoencoder와 resnet을 동시에 학습.
- 이 과정에서 reconstruction loss와 prediction loss를 합산하여 backpropagation한다. 

## Autoencoder training

여기서는 Autoencoder와 Resnet을 한 시스템으로 묶어버리도록 하겠다. 

In [10]:

_, autoencoder, resnet = train_auto_resnet_chain(config, torch.utils.data.ConcatDataset([train_dataset, val_dataset]),
                                                 lamda = 0.01, validation_dataset=None, use_momentum=True, callback_epoch=None)

metrics = torchmetrics.MetricTracker(torchmetrics.MetricCollection(
    {"R_cellwise_residuals":scripts.GroupwiseMetric(metric=torchmetrics.functional.pearson_corrcoef,
                          grouping="drugs",
                          average="macro",
                          residualize=True),
    "R_cellwise":scripts.GroupwiseMetric(metric=torchmetrics.functional.pearson_corrcoef,
                          grouping="cell_lines",
                          average="macro",
                          residualize=False),
    "MSE":torchmetrics.MeanSquaredError()}))
device = torch.device(config["env"]["device"])
metrics.to(device)
test_dataloader = torch.utils.data.DataLoader(test_dataset,
                                       batch_size=config["optimizer"]["batch_size"],
                                       drop_last=False,
                                      shuffle=False,
                                      pin_memory=True)

/home/kim14/.conda/envs/jupyter/lib/python3.10/site-packages/torch/nn/modules/lazy.py:180: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '
/home/kim14/.conda/envs/jupyter/lib/python3.10/site-packages/torchmetrics/utilities/prints.py:43: FutureWarning: The default value for `maximize` will be changed from `True` to `None` in v1.7.0 of TorchMetrics,will automatically infer the value based on the `higher_is_better` attribute of the metric (if such attribute exists) or raise an error if it does not. If you are explicitly setting the `maximize` argument to either `True` or `False` already, you can ignore this warning.
  warnings.warn(*args, **kwargs)  # noqa: B028


Epoch 1/100 | Train Loss: 20.51217029206564 | Validation R: None
Epoch 2/100 | Train Loss: 16.460548546656835 | Validation R: None
Epoch 3/100 | Train Loss: 15.217209692574666 | Validation R: None
Epoch 4/100 | Train Loss: 12.824020484850204 | Validation R: None
Epoch 5/100 | Train Loss: 8.373624313611788 | Validation R: None
Epoch 6/100 | Train Loss: 4.5565076613705875 | Validation R: None
Epoch 7/100 | Train Loss: 3.198260305564075 | Validation R: None
Epoch 8/100 | Train Loss: 2.7481189669926493 | Validation R: None
Epoch 9/100 | Train Loss: 2.5187890209760133 | Validation R: None
Epoch 10/100 | Train Loss: 2.3790631753957863 | Validation R: None
Epoch 11/100 | Train Loss: 2.3007810707490814 | Validation R: None
Epoch 12/100 | Train Loss: 2.254846773364327 | Validation R: None
Epoch 13/100 | Train Loss: 2.2259324644842455 | Validation R: None
Epoch 14/100 | Train Loss: 2.198489167997914 | Validation R: None
Epoch 15/100 | Train Loss: 2.1735234575187707 | Validation R: None
Epoch 16/

# Evaluation

트레이닝된 Autoencoder의 encoder와, Resnet을 하나의 시스템으로 사용한다. 

In [11]:
autoencoder_resnet_chain = evaluate_step(autoencoder, resnet, test_dataloader, metrics, device,
                                        save_predictions=True, model_name = "auto-resnet-chain", dataset_name = "raw_0.01")
print(f"main model final metrics: {autoencoder_resnet_chain}")
#2907s

/work/kim14/project_work/scripts/models.py:69: UserWarning: The use of `x.T` on tensors of dimension other than 2 to reverse their shape is deprecated and it will throw an error in a future release. Consider `x.mT` to transpose batches of matrices or `x.permute(*torch.arange(x.ndim - 1, -1, -1))` to reverse the dimensions of a tensor. (Triggered internally at /opt/conda/conda-bld/pytorch_1670525552843/work/aten/src/ATen/native/TensorShape.cpp:3277.)
  return torch.linalg.solve(A, Xy).T


Predictions saved to: results/pred_auto-resnet-chain_raw_0.01_20250316_02:04:59_4bec1760-84bb-4c8c-97ac-862d84fadb04.csv
main model final metrics: {'MSE': 1.6427305936813354, 'R_cellwise': 0.8919653296470642, 'R_cellwise_residuals': 0.27920225262641907}


# Hyperparameter Optimization

In [23]:
import optuna

def train_model_optuna(trial, config):
    """
    Optuna를 활용한 하이퍼파라미터 최적화 함수
    - Autoencoder + ResNet을 함께 최적화
    """

    ### 1️⃣ 하이퍼파라미터 샘플링 ###
    # Autoencoder 관련
    config["model"]["embed_dim"] = trial.suggest_int("embed_dim", 64, 512)
    config["model"]["hidden_dim_encoders"] = trial.suggest_int("hidden_dim_encoders", 64, 1024)
    config["model"]["fusion_dim"] = trial.suggest_int("fusion_dim", 64, 1024)
    config["model"]["dropout_encoders"] = trial.suggest_float("dropout_encoders", 0.1, 0.5)
    config["model"]["pre_training_epochs"] = trial.suggest_int("pre_training_epochs", 10, 200)

    # ResNet 관련
    config["model"]["hidden_dim"] = trial.suggest_int("hidden_dim", 256, 2048)
    config["model"]["dropout"] = trial.suggest_float("dropout", 0.1, 0.5)
    config["model"]["n_layers"] = trial.suggest_int("n_layers", 2, 10)
    config["model"]["dropout_omics"] = trial.suggest_float("dropout_omics", 0.1, 0.9)
    config["model"]["dropout_omics_finetuning"] = trial.suggest_float("dropout_omics_finetuning", 0.1, 0.9)

    # Optimizer 관련
    config["optimizer"]["batch_size"] = trial.suggest_int("batch_size", 64, 512)
    config["optimizer"]["clip_norm"] = trial.suggest_int("clip_norm", 5, 20)
    config["optimizer"]["learning_rate"] = trial.suggest_float("learning_rate", 1e-6, 1e-2, log=True)
    config["optimizer"]["lr_pretraining"] = trial.suggest_float("lr_pretraining", 1e-6, 1e-2, log=True)
    config["optimizer"]["pre_batch_size"] = trial.suggest_int("pre_batch_size", 64, 512)

    ### 2️⃣ 데이터 로딩 ###
    #train_dataset, val_dataset, test_dataset, input_dims = get_data_corr(
    #    n_fold=config["env"]["fold"],
    #    fp_radius=config["features"]["fp_radius"],
    #    transform_into_corr=config["features"]["use_correlation_representation"],
    #    typ=("rnaseq", "proteomics", "mutations", "methylations")
    #)

    ### 3️⃣ 모델 학습 (Autoencoder + ResNet 동시 학습) ###
    try:
        val_target, autoencoder, resnet = train_auto_resnet_chain(
            config, train_dataset, val_dataset, use_momentum=True
        )
        print(f"Trial {trial.number}: Validation R_cellwise_residuals = {val_target}")

        return val_target  # 최적화 목표 (Validation R값 최대화)

    except Exception as e:
        print(f"Error in trial {trial.number}: {e}")
        return -float("inf")  # 실패 시 최소 값 반환


In [24]:
#if config["env"]["search_hyperparameters"]:
true = True
if true:
    study_name = "auto_resnet_chain_opt"
    storage_name = f"sqlite:///studies/{study_name}.db"

    study = optuna.create_study(
        study_name=study_name,
        storage=storage_name,
        direction='maximize',  # Validation R 값을 최대화
        load_if_exists=True,
        pruner=optuna.pruners.MedianPruner(n_startup_trials=30, n_warmup_steps=5, interval_steps=5)
    )

    study.optimize(lambda trial: train_model_optuna(trial, config), n_trials=40)

    best_config = study.best_params
    print("Best Hyperparameters:", best_config)

    # 최적의 하이퍼파라미터를 config에 반영
    config["model"]["embed_dim"] = best_config["embed_dim"]
    config["model"]["hidden_dim_encoders"] = best_config["hidden_dim_encoders"]
    config["model"]["fusion_dim"] = best_config["fusion_dim"]
    config["model"]["dropout_encoders"] = best_config["dropout_encoders"]
    config["model"]["pre_training_epochs"] = best_config["pre_training_epochs"]
    config["model"]["hidden_dim"] = best_config["hidden_dim"]
    config["model"]["dropout"] = best_config["dropout"]
    config["model"]["n_layers"] = best_config["n_layers"]
    config["model"]["dropout_omics"] = best_config["dropout_omics"]
    config["model"]["dropout_omics_finetuning"] = best_config["dropout_omics_finetuning"]
    config["optimizer"]["batch_size"] = best_config["batch_size"]
    config["optimizer"]["clip_norm"] = best_config["clip_norm"]
    config["optimizer"]["learning_rate"] = best_config["learning_rate"]
    config["optimizer"]["lr_pretraining"] = best_config["lr_pretraining"]
    config["optimizer"]["pre_batch_size"] = best_config["pre_batch_size"]



[I 2025-03-12 23:06:27,799] Using an existing study with name 'auto_resnet_chain_opt' instead of creating a new one.


Epoch 1/100 | Train Loss: 5.489410892630046 | Validation R: 0.01004643552005291
Epoch 2/100 | Train Loss: 2.5610766532146823 | Validation R: 0.003304628189653158
Epoch 3/100 | Train Loss: 2.2855825964987804 | Validation R: 0.0018826914951205254
Epoch 4/100 | Train Loss: 2.1744253857407423 | Validation R: -0.0051840155199170115
Epoch 5/100 | Train Loss: 2.1126591061002915 | Validation R: -0.007814185746014118
Epoch 6/100 | Train Loss: 2.076366836729303 | Validation R: -0.014115806306898595
Epoch 7/100 | Train Loss: 2.0563792280127515 | Validation R: -0.009512321578562261
Epoch 8/100 | Train Loss: 2.0433153385082177 | Validation R: -0.009597727194845678
Epoch 9/100 | Train Loss: 2.032511304450774 | Validation R: -0.00922377577134371
Epoch 10/100 | Train Loss: 2.0214865629322234 | Validation R: -0.007527938079254628
Epoch 11/100 | Train Loss: 2.0143513717054002 | Validation R: -0.012388687201622487
Epoch 12/100 | Train Loss: 2.016638361956034 | Validation R: -0.014613377227044774
Epoch 13

[W 2025-03-12 23:24:30,230] Trial 163 failed with parameters: {'embed_dim': 508, 'hidden_dim_encoders': 812, 'fusion_dim': 596, 'dropout_encoders': 0.26815411194064864, 'pre_training_epochs': 175, 'hidden_dim': 1994, 'dropout': 0.29737656926914646, 'n_layers': 5, 'dropout_omics': 0.856976151498954, 'dropout_omics_finetuning': 0.5516169494497061, 'batch_size': 280, 'clip_norm': 10, 'learning_rate': 3.068319236416493e-06, 'lr_pretraining': 1.4284574022734964e-06, 'pre_batch_size': 114} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/home/kim14/.conda/envs/jupyter/lib/python3.10/site-packages/optuna/study/_optimize.py", line 197, in _run_trial
    value_or_values = func(trial)
  File "/tmp/ipykernel_175440/4219314737.py", line 15, in <lambda>
    study.optimize(lambda trial: train_model_optuna(trial, config), n_trials=40)
  File "/tmp/ipykernel_175440/944425779.py", line 41, in train_model_optuna
    val_target, autoencoder, resnet = train_

KeyboardInterrupt: 

In [ ]:
# 2️⃣ 최적 하이퍼파라미터를 적용한 후 Training 실행
train_dataset, val_dataset, test_dataset, input_dims = get_data_corr(
    n_fold=config["env"]["fold"],
    fp_radius=config["features"]["fp_radius"],
    transform_into_corr=config["features"]["use_correlation_representation"],
    typ=("rnaseq", "proteomics", "mutations", "methylations")
)

val_target, autoencoder, resnet = train_auto_resnet_chain(config, train_dataset, val_dataset, use_momentum=True)

print(f"Final Training Complete! Final R_cellwise_residuals: {val_target}")

# Cross validation

In [27]:
from rdkit import RDLogger

RDLogger.DisableLog('rdApp.warning')  # RDKit의 경고 메시지 제거
RDLogger.DisableLog('rdApp.error') 
warnings.simplefilter("ignore", DeprecationWarning)

In [31]:
lambda_candidates = [0.001, 0.01, 0.1, 0.3, 0.5, 0.7, 0.9, 0.99, 0.999]

# 10-Fold Cross Validation 수행
# K-Fold 분할 (10-Fold)
folds = 10
cv_results = {}

for lambda_value in lambda_candidates:
    print(f"====== Optimizing for Lambda: {lambda_value} ======")
    fold_results = []

    for fold in range(folds):
        # 데이터셋을 각 폴드별로 분할
        train_dataset, val_dataset, test_dataset, input_dims = get_data_corr(n_fold=fold, transform_into_corr=False, 
                                                                             typ = ("rnaseq", "proteomics", "mutations", "methylations"))

        _, autoencoder, resnet = train_auto_resnet_chain(config, torch.utils.data.ConcatDataset([train_dataset, val_dataset]),
                                                 lamda = lambda_value, validation_dataset=None, use_momentum=True, callback_epoch=None)

        metrics = torchmetrics.MetricTracker(torchmetrics.MetricCollection(
            {"R_cellwise_residuals":scripts.GroupwiseMetric(metric=torchmetrics.functional.pearson_corrcoef,
                                  grouping="drugs",
                                  average="macro",
                                  residualize=True),
            "R_cellwise":scripts.GroupwiseMetric(metric=torchmetrics.functional.pearson_corrcoef,
                                  grouping="cell_lines",
                                  average="macro",
                                  residualize=False),
            "MSE":torchmetrics.MeanSquaredError()}))
        device = torch.device(config["env"]["device"])
        metrics.to(device)
        test_dataloader = torch.utils.data.DataLoader(test_dataset,
                                               batch_size=config["optimizer"]["batch_size"],
                                               drop_last=False,
                                              shuffle=False,
                                              pin_memory=True)
        autoencoder_resnet_chain = evaluate_step(autoencoder, resnet, test_dataloader, metrics, device)

        fold_results.append(autoencoder_resnet_chain['R_cellwise_residuals'])

    # 각 Lambda에 대해 폴드 평균 성능 계산
    cv_results[lambda_value] = np.mean(fold_results)

# 최적의 Lambda 값 출력
best_lambda = max(cv_results, key=cv_results.get)
print(f"Best Lambda: {best_lambda} with average R_cellwise_residuals: {cv_results[best_lambda]}")


====== Optimizing for Lambda: 0.001 ======


[03:13:33] DEPRECATION WARNING: please use MorganGenerator
[03:13:33] DEPRECATION WARNING: please use MorganGenerator
[03:13:33] DEPRECATION WARNING: please use MorganGenerator
[03:13:33] DEPRECATION WARNING: please use MorganGenerator
[03:13:33] DEPRECATION WARNING: please use MorganGenerator
[03:13:33] DEPRECATION WARNING: please use MorganGenerator
[03:13:33] DEPRECATION WARNING: please use MorganGenerator
[03:13:33] DEPRECATION WARNING: please use MorganGenerator
[03:13:33] DEPRECATION WARNING: please use MorganGenerator
[03:13:33] DEPRECATION WARNING: please use MorganGenerator
[03:13:33] DEPRECATION WARNING: please use MorganGenerator
[03:13:33] DEPRECATION WARNING: please use MorganGenerator
[03:13:33] DEPRECATION WARNING: please use MorganGenerator
[03:13:33] DEPRECATION WARNING: please use MorganGenerator
[03:13:33] DEPRECATION WARNING: please use MorganGenerator
[03:13:33] DEPRECATION WARNING: please use MorganGenerator
[03:13:33] DEPRECATION WARNING: please use MorganGenerat

Epoch 1/100 | Train Loss: 20.708787205282196 | Validation R: None


KeyboardInterrupt: 

# 저장

In [33]:
result = autoencoder_resnet_chain

model_name = "auto-resnet-chain-raw0.65"
result["Model"] = model_name
time = datetime.now().strftime("%Y%m%d_%H:%M")
result["Time"] = time

result_df = pd.DataFrame([result])

ev_table = pd.read_csv("results/evaluation_table.csv")
ev_table = pd.concat([ev_table, result_df], ignore_index=True)
#ev_table = ev_table.drop_duplicates(subset=["Model"],keep = "last")
display(ev_table)

,MSE,R_cellwise,R_cellwise_residuals,Model,Time
0,1.795264,0.890693,0.333604,baseline_rnaseq,20241206_19:25
1,1.977368,0.874748,0.238710,baseline_proteomics,20241206_19:26
2,2.400242,0.866647,0.093873,baseline_mutations,20241206_19:26
3,2.243853,0.868511,0.167871,baseline_methylations,20241206_19:26
4,2.230996,0.868163,0.138434,baseline_cnvs,20241206_19:26
5,1.810338,0.886230,0.320672,baseline_concat,20241206_19:26
6,1.646177,0.897545,0.327031,baseline_ensemble,20241206_19:26
7,1.882182,0.884252,0.265135,encoder_sub-network,20241206_19:27
8,2.265149,0.876332,-0.011226,auto-resnet-chain-sim,20250314_02:16
9,1.594700,0.894793,0.300049,auto-resnet-chain-raw,20250314_02:23


In [34]:
ev_table.to_csv("results/evaluation_table.csv", index = False)

In [25]:
ev_table['Model'][20] = 'auto-resnet-chain-raw0.3'

/tmp/ipykernel_1275509/86570976.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ev_table['Model'][20] = 'auto-resnet-chain-raw0.3'
